In [58]:
import pyfiles.HFSparser as HFSparser
import pandas as pd
import matplotlib.pyplot as plt

df = HFSparser.directory_to_df()
df

,Nk,ndim,rs,deltaK,kf,kmax,Nocc,Nvir,Nexc,dav_its,Dav_Lowest_Val,Occ Energies,Vir Energies,Kgrid,All Davidson Eigenvalues at Last Iteration,Davidson lowest eigenvalues at each iteration,Occupied States,Virtual States,Excitations
test1,11,2,1.2,0.471405,1.178511,2.357023,21,79,117,8,0.111856,"[-0.0686323805205, -0.233601864085, -0.0686323...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[-2.35702260396, -1.88561808316, -1.4142135623...","[0.111856109137, 0.173802170042, 0.17380217005...","[0.321551633699, 0.23730703994, 0.116337773186...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [..."
test2,11,2,1.2,0.471405,1.178511,2.357023,21,79,117,8,0.111856,"[-0.0686323805205, -0.233601864085, -0.0686323...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[-2.35702260396, -1.88561808316, -1.4142135623...","[0.111856109137, 0.173802170042, 0.17380217005...","[0.321551633699, 0.23730703994, 0.116337773186...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [..."
test4,11,2,1.2,0.471405,1.178511,2.357023,21,79,117,8,0.111856,"[-0.0686323805205, -0.233601864085, -0.0686323...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[-2.35702260396, -1.88561808316, -1.4142135623...","[0.111856109137, 0.173802170042, 0.17380217005...","[0.321551633699, 0.23730703994, 0.116337773186...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [..."
test3,11,2,1.2,0.471405,1.178511,2.357023,21,79,117,8,0.111856,"[-0.0686323805205, -0.233601864085, -0.0686323...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[-2.35702260396, -1.88561808316, -1.4142135623...","[0.111856109137, 0.173802170042, 0.17380217005...","[0.321551633699, 0.23730703994, 0.116337773186...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [..."
